How to use the Twitter API and Py-ART to Tweet Your Radar!
----------------------------------------------------------

*Scott Collis, Argonne National Laboratory*

In [7]:
#First some imports
import twitter #https://python-twitter.readthedocs.io/en/latest/index.html
import json
import pyart #https://github.com/ARM-DOE/pyart
import matplotlib.pyplot as plt
from boto.s3.connection import S3Connection #Anaconda installable
import shutil, os
from datetime import timedelta, datetime
import numpy as np
import tempfile
import shutil, os

First, some functions to deal with Amazon Web Services Simple Storage Service (s3) holding of NOAA, NWS NEXRAD. This is an **amazing** set up as part of the CRADA between NOAA, Amazon and Unidata. https://aws.amazon.com/noaa-big-data/nexrad/

In [8]:
def nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))


def get_radar_from_aws(site, datetime_t):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation 
    datetime_t : datetime
        desired date time
    """
    
    #First create the query string for the bucket knowing
    #how NOAA and AWS store the data
    
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site
    
    #Connect to the bucket
    
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')
    
    #Get a list of files 
    
    bucket_list = list(bucket.list(prefix = my_pref))
    #print(bucket_list)
    #we are going to create a list of keys and datetimes to allow easy searching
    
    keys = []
    datetimes = []
    
    #populate the list

    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-3]
            fmt = '%Y%m%d_%H%M%S_V06' 
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])
            #print(dt)
        if this_str[-3::] == 'V06': #'LSX20160707_000150_' does not match format '%Y%m%d_%H%M%S_V06'
            #print(this_str)
            #print(this_str[-19::])
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06' 
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])
    
    #function to allow easy searching 
    
    def func(x):
        delta =  x - datetime_t if x > datetime_t else timedelta.max
        return delta
    
    #find the closest available radar to your datetime 
    
    closest_datetime = nearestDate(datetimes, datetime_t)
    index = datetimes.index(closest_datetime)
    #print(closest_datetime)
    #create a temp file, download radar data to file from S3
    #read into a radar object and return
    
    localfile = tempfile.NamedTemporaryFile()
    keys[index].get_contents_to_filename(localfile.name)
    radar = pyart.io.read(localfile.name)
    return radar

Ok.. now the part that takes some work.. Take a loom here:

https://python-twitter.readthedocs.io/en/latest/getting_started.html

This gives you instruction on how to populate the json file... 

Here is an example:
`
➜  twitterradar git:(tools_init) ✗ more token/PyWeather.json 
{"consumer_key":"SOMETHING",
"consumer_secret":"SOMETHINGELSE",
"access_token_key":"ANOTHERTHING",
"access_token_secret":"YETANOTHERTHING"}
`



In [2]:
fh = open('/Users/scollis/projects/TwitterRadar/twitterradar/token/PyWeather.json')
myson = json.load(fh)
fh.close()

Initialize the API

In [3]:
api = twitter.Api(consumer_key=myson['consumer_key'],
                  consumer_secret=myson['consumer_secret'],
                  access_token_key=myson['access_token_key'],
                  access_token_secret=myson['access_token_secret'])

Lets try posting!

In [5]:
status = api.PostUpdate("""Hello World, once againthis is a test 
                      of tweeting using the Python API""")


In [6]:
print(status)

{"created_at": "Tue Oct 18 00:25:39 +0000 2016", "hashtags": [], "id": 788174166906118145, "id_str": "788174166906118145", "lang": "en", "source": "<a href=\"https://github.com/scollis/\" rel=\"nofollow\">PythonWeatherRobot</a>", "text": "Hello World, once againthis is a test \n                      of tweeting using the Python API", "urls": [], "user": {"created_at": "Mon Oct 17 21:13:20 +0000 2016", "default_profile": true, "default_profile_image": true, "followers_count": 1, "id": 788125768974999553, "lang": "en", "name": "Python Weather Robot", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_0_normal.png", "profile_link_color": "2B7BB9", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "PyWeather", "statuses_count": 4}, "user_mentions": []}


Ok.. Now a cool method for tweeting your radar!

In [36]:
def tweet_my_radar(datetime, radar, tapi, min_lat = None,
                  max_lat = None, min_lon = None, 
                  max_lon = None):
    """
    Fetch a radar from S3, plot it and tweet plus statistics.
    
    Grab a radar from a site and use the Twitter API
    to tweet the PPI from the lowest tilt to twitter.
    Also tweet the number of gates above two reflectivity
    thresholds and the min and max reflectivity.
    
    Parameters
    ----------
    datetime: datetime object
        Python datetime object to be passed to the method
        to find the nearest radar object from AWS S3 using
        boto. 
    
    radar: String
        Four letter radar code. eg KLOT
    
    min_lat, max_lat, min_lon, max_lon: floats
        bounds for the display
        
    Returns
    -------
    None
    
    """
    #Get a Py-ART radar Object
    my_radar = get_radar_from_aws(radar,b_d )
    #Make a display
    display = pyart.graph.RadarMapDisplay(my_radar)
    fig = plt.figure(figsize = [10,8])
    #Plot Z from lowest tilt
    display.plot_ppi_map('reflectivity', sweep = 0, resolution = 'i',
                        vmin = -8, vmax = 64, mask_outside = False,
                        cmap = pyart.graph.cm.NWSRef,
                        min_lat = min_lat, min_lon = min_lon,
                        max_lat = max_lat, max_lon = max_lon)
    #get a tempfile
    localfile = tempfile.NamedTemporaryFile()
    #Save to tempfile.. Need png or Twitter gets grumpy 
    plt.savefig(localfile.name + '.png')
    
    #Now grab some statistics.. 
    min_z = my_radar.fields['reflectivity']['data'].min()
    max_z = my_radar.fields['reflectivity']['data'].max()
    n_gates_20 = len(np.where(my_radar.fields['reflectivity']['data'] > 20.0)[0])
    n_gates_40 = len(np.where(my_radar.fields['reflectivity']['data'] > 40.0)[0])
    
    #Format the strings.. TODO: turn into percent
    gdata = "There are {0} gates above 20dBZ and {1} above 40dBZ".format(n_gates_20, 
                                                                     n_gates_40)
    
    #Make the tweet text
    mmdata = "The min Z is {0}dBZ and the max is {1}dBZ".format(min_z,
                                                            max_z)
    #And... post it.. yes.. it is that easy!
    tapi.PostUpdate( gdata + ' ' + mmdata, 
              media = localfile.name + '.png')
    

In [37]:
#Look at some special cases.

base_date = "20110520_100000"
fmt = '%Y%m%d_%H%M%S' 
b_d = datetime.strptime(base_date, fmt)
min_lat = 35
max_lat = 38.5
min_lon = -100
max_lon = -96.5

tweet_my_radar(b_d, 'KVNX', api, min_lat = min_lat,
                  max_lat = max_lat, min_lon = min_lon, 
                  max_lon = max_lon)

In [41]:
#Tweet the latest Chicago Image!
max_lat = 42.5 
min_lat = 41
min_lon = -88.5 
max_lon = -86
b_d = datetime.utcnow()
tweet_my_radar(b_d, 'KLOT', api, min_lat = min_lat,
                  max_lat = max_lat, min_lon = min_lon, 
                  max_lon = max_lon)

In [42]:
#And from the ARM site 
min_lat = 35
max_lat = 38.5
min_lon = -100
max_lon = -96.5

tweet_my_radar(b_d, 'KVNX', api, min_lat = min_lat,
               max_lat = max_lat, min_lon = min_lon, 
                  max_lon = max_lon)
